In [1]:
from pathlib import Path  # optional, for nice path specifications

import pprint as pp  # optional, for nice printing
import numpy as np  # numerics library
import matplotlib.pyplot as plt  # plotting capabilities
import pandas as pd

import ard  # technically we only really need this
from ard.utils.io import load_yaml  # we grab a yaml loader here
from ard.api import set_up_ard_model  # the secret sauce
from ard.viz.layout import plot_layout  # a plotting tool!

import openmdao.api as om  # for N2 diagrams from the OpenMDAO backend

%matplotlib inline

In [2]:
# load input
path_inputs = Path.cwd().absolute() / "inputs"
input_dict = load_yaml(path_inputs / "ard_frankenstein.yaml")

# create and setup system
prob = set_up_ard_model(input_dict=input_dict, root_data_path=path_inputs)

Adding top_level
Adding gridfarm
Adding boundary
Adding exclusions
Adding spacing_constraint


In [3]:
prob.run_driver()

Problem: problem
Driver:  DOEDriver
  success     : True
  iterations  : 50000
  runtime     : 5.7779E+01 s
  model_evals : 50000
  model_time  : 2.1103E+01 s
  deriv_evals : 0
  deriv_time  : 0.0000E+00 s
  exit_status : SUCCESS

In [9]:
# Access the recorder data
case_reader = om.CaseReader(prob.get_outputs_dir() / "cases.sql")

# Get all driver cases
driver_cases = case_reader.list_cases("driver", out_stream=None)

# Extract data from all cases
results = []
for index_case, case_id in enumerate(driver_cases):

    case = case_reader.get_case(case_id)

    result = {
        "case_id": case_id,
        "angle_orientation": float(case.get_val("angle_orientation")[0]),
        "spacing_primary": float(case.get_val("spacing_primary")[0]),
        "angle_skew": float(case.get_val("angle_skew")[0]),
        "spacing_secondary": float(case.get_val("spacing_secondary")[0]),
        "max_boundary_distance": float(np.max(case.get_val("boundary_distances"))),
        "boundary_satisfying": np.all(np.max(case.get_val("boundary_distances")) <= 0.0),
        "max_exclusion_distance": float(np.max(case.get_val("exclusion_distances"))),
        "exclusion_satisfying": np.all(np.max(case.get_val("exclusion_distances")) <= 0.0),
        "turbine_spacing_satisfying": np.all(case.get_val("spacing_constraint.turbine_spacing", units="km") >= 0.552),
        "x_turbines": case.get_val("x_turbines"),
        "y_turbines": case.get_val("y_turbines"),
    }
    results.append(result)



results = pd.DataFrame(results)
results

,case_id,angle_orientation,spacing_primary,angle_skew,spacing_secondary,max_boundary_distance,boundary_satisfying,max_exclusion_distance,exclusion_satisfying,turbine_spacing_satisfying,x_turbines,y_turbines
0,rank0:DOEDriver_LatinHypercube|0,43.117420,7.362764,-29.799976,6.251130,628.199667,False,-29.306325,True,True,"[-535.1264437847808, 119.08784828972239, 773.3...","[436.5933692140518, 1135.2760989250046, 1833.9..."
1,rank0:DOEDriver_LatinHypercube|1,63.498734,13.799778,41.080567,12.317818,2213.367716,False,-232.506532,True,True,"[-40.337800630846004, 1565.1309430943984, 3170...","[2175.0193952812915, 2975.5208737514145, 3776...."
2,rank0:DOEDriver_LatinHypercube|2,153.923221,14.229969,-44.972852,4.236872,1085.654123,False,-11.039665,True,True,"[470.0583414137956, 1283.2266821104065, 2096.3...","[3317.5208095966454, 1655.9335593629257, -5.65..."
3,rank0:DOEDriver_LatinHypercube|3,119.063337,7.879951,-22.636018,4.688219,339.533492,False,-118.337343,True,True,"[208.90107421518348, 1104.3063617722169, 1999....","[2073.371868789938, 1575.7458759912429, 1078.1..."
4,rank0:DOEDriver_LatinHypercube|4,172.483102,10.473715,1.600890,7.814066,728.462950,False,-177.667193,True,True,"[1863.079161172218, 2041.1995059544563, 2219.3...","[2374.185913574902, 1024.304005544126, -325.57..."
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,rank0:DOEDriver_LatinHypercube|49995,127.865085,3.827995,7.491656,6.451292,128.273109,False,130.060015,False,False,"[1239.367170679271, 1632.2327025869909, 2025.0...","[1819.4121745429509, 1513.959007204571, 1208.5..."
49996,rank0:DOEDriver_LatinHypercube|49996,102.463714,3.614425,1.451333,12.500468,469.784583,False,122.857292,False,False,"[962.5098669716621, 1421.3114959891698, 1880.1...","[2598.847210119348, 2497.4381355499627, 2396.0..."
49997,rank0:DOEDriver_LatinHypercube|49997,177.947314,5.334341,14.755355,9.235750,697.437087,False,150.384131,False,True,"[2216.751525033408, 2241.5903553373555, 2266.4...","[1339.564793221956, 646.545390882799, -46.4740..."
49998,rank0:DOEDriver_LatinHypercube|49998,70.892680,6.378826,14.210702,11.884311,785.000504,False,-232.506532,True,True,"[110.78720674561703, 894.3489213793423, 1677.9...","[2236.0292577233186, 2507.473941144228, 2778.9..."


In [10]:
results[results.boundary_satisfying & results.turbine_spacing_satisfying]

,case_id,angle_orientation,spacing_primary,angle_skew,spacing_secondary,max_boundary_distance,boundary_satisfying,max_exclusion_distance,exclusion_satisfying,turbine_spacing_satisfying,x_turbines,y_turbines
98,rank0:DOEDriver_LatinHypercube|98,4.322342,6.331174,17.902556,4.165854,-25.262635,True,176.099645,False,True,"[441.51967987703097, 503.55116224157007, 565.5...","[314.1141575334741, 1134.8259136079837, 1955.5..."
102,rank0:DOEDriver_LatinHypercube|102,103.519970,5.008005,8.520614,6.657907,-0.916619,True,96.351983,False,True,"[725.8131015447784, 1358.8123878103113, 1991.8...","[1882.9916881909248, 1730.7886434496586, 1578...."
643,rank0:DOEDriver_LatinHypercube|643,17.460949,4.990256,12.885828,4.162992,-194.401586,True,145.334875,False,True,"[356.6278586693903, 551.2840041949662, 745.940...","[581.2103615891912, 1200.0511573938718, 1818.8..."
905,rank0:DOEDriver_LatinHypercube|905,99.753598,4.747296,5.023813,6.729700,-134.898735,True,97.025110,False,True,"[646.1653027692987, 1254.3932226800246, 1862.6...","[1873.2995714739131, 1768.7475745644056, 1664...."
1211,rank0:DOEDriver_LatinHypercube|1211,94.497960,4.246709,-1.780868,4.424870,-88.911211,True,60.245027,False,True,"[507.2965264437157, 1057.668412925124, 1608.04...","[1537.720726755957, 1494.425238518314, 1451.12..."
...,...,...,...,...,...,...,...,...,...,...,...,...
49580,rank0:DOEDriver_LatinHypercube|49580,5.999055,7.446525,-12.554872,4.214911,-14.436132,True,72.146455,False,True,"[371.5230880114243, 472.695799938102, 573.8685...","[-107.27690369887091, 855.4699589051128, 1818...."
49601,rank0:DOEDriver_LatinHypercube|49601,28.162043,6.080311,-31.489968,3.901676,-14.640468,True,106.502169,False,True,"[63.51263663021041, 436.57429941032126, 809.63...","[188.16663844106597, 885.0318289604253, 1581.8..."
49714,rank0:DOEDriver_LatinHypercube|49714,104.908012,4.780246,8.896410,4.750243,-40.951536,True,125.182784,False,True,"[682.1525448032633, 1282.6672432652922, 1883.1...","[1651.312904270587, 1491.4383758073525, 1331.5..."
49828,rank0:DOEDriver_LatinHypercube|49828,97.432739,4.767936,4.601716,5.759782,-100.128158,True,52.984766,False,True,"[572.3868605058601, 1187.0102990466494, 1801.6...","[1734.4277066401223, 1654.2449074880121, 1574...."


In [13]:
results.to_csv("structured_layout_samples.csv", index=False)